In [2]:
import pickle
import numpy
import plotly
from collections import defaultdict
import plotly.graph_objs as go

plotly.offline.init_notebook_mode(connected=True)

### Plotter Functions

In [71]:
def processData(x,y,z):
    # Get x,y,z sorted based on y i.e TXF
    sortxyz = [(x,y,z) for y,x,z in sorted(zip(y,x,z))]
    
    x = [i[0] for i in sortxyz]
    y = [i[1] for i in sortxyz]
    z = [i[2] for i in sortxyz]
    
    return x,y,z
    
    
def plot3DWithTXF(x,y,z,xtitle, ytitle, ztitle):
    
    colors = ['#e6194b', '#3cb44b', '#ffe119', '#4363d8', '#f58231', '#911eb4']
    
    # Get x,y,z sorted based on y i.e TXF
    x,y,z = processData(x,y,z)
    
    txDictx = defaultdict(list)
    txDictz = defaultdict(list)
    
    for i in range(len(y)):
        txDictx[y[i]].append(x[i])
        txDictz[y[i]].append(z[i])

    data = []   
         
    cnt = 0          
    for key in txDictx.keys():
        
        x_d = txDictx[key]
        y_d = [key for i in range(len(txDictx[key]))]
        z_d = txDictz[key]

        temp = go.Scatter3d(x=x_d, 
                         y=y_d, 
                         z=z_d,
                        line=dict(
                            color=colors[cnt % len(colors)],
                            width=2
                        ) 
                    )
        cnt+=1      
        data.append(temp)
    
    fig = go.Figure(data = data)
    fig.update_layout(scene=dict(
        xaxis_title=xtitle,
        yaxis_title=ytitle,
        zaxis_title=ztitle),
    )

    plotly.offline.iplot(fig)
    
def plotWithCDP(cdp_axis, txf_axis, dai_price_history_axis, xtitle, ytitle, ztitle):
    
    colors = ['#e6194b', '#3cb44b', '#ffe119', '#4363d8', '#f58231', '#911eb4']
    
    data = []
    cnt = 0
    
    # generate an axis with numbers 1 -> number of days
    days_axis = [i for i in range(1, len(dai_price_history_axis[0]) + 1)]
    
    for k in range(0, len(dai_price_history_axis)):
        x_axis = [cdp_axis[k] for i in range(0, len(days_axis))]
        y_axis = days_axis
        z_axis = dai_price_history_axis[k]
        
        temp = go.Scatter3d(x=x_axis, 
                            y=y_axis, 
                            z=z_axis,
                            name = "tx_fee: " + str(txf_axis[k]) + " cdp_rate: " + str(cdp_axis[k]),
                            line=dict(
                            color=colors[cnt % len(colors)],
                            width=1
                           ) 
                        )
        
        cnt+=1      
        data.append(temp)
    
    fig = go.Figure(data = data)
    fig.update_layout(scene=dict(
        xaxis_title=xtitle,
        yaxis_title=ytitle,
        zaxis_title=ztitle),
    )

    plotly.offline.iplot(fig)

def plotWithTXF(txf_axis, cdp_axis, dai_price_history_axis, xtitle, ytitle, ztitle):
    
    colors = ['#e6194b', '#3cb44b', '#ffe119', '#4363d8', '#f58231', '#911eb4']
    
    data = []
    cnt = 0
    
    # generate an axis with numbers 1 -> number of days
    days_axis = [i for i in range(1, len(dai_price_history_axis[0]) + 1)]
    
    for k in range(0, len(dai_price_history_axis)):
        x_axis = [txf_axis[k] for i in range(0, len(days_axis))]
        y_axis = days_axis
        z_axis = dai_price_history_axis[k]
        
        temp = go.Scatter3d(x=x_axis, 
                            y=y_axis, 
                            z=z_axis,
                            name = "cdp_rate: " + str(cdp_axis[k]) + " tx_fee: " + str(txf_axis[k]),
                            line=dict(
                            color=colors[cnt % len(colors)],
                            width=1
                           ) 
                        )
        
        cnt+=1      
        data.append(temp)
    
    fig = go.Figure(data = data)
    fig.update_layout(scene=dict(
        xaxis_title=xtitle,
        yaxis_title=ytitle,
        zaxis_title=ztitle),
    )

    plotly.offline.iplot(fig)      

### Plot final settling DAI Price with TXF and CDPRate

In [72]:
filename = "test1/sim-summary.pickle"
input_file = open(filename,'rb')
cdp_axis, txf_axis, dai_price_history_axis = pickle.load(input_file)
dai_axis = [dai_price_history[-1] for dai_price_history in dai_price_history_axis]
input_file.close()

In [73]:
plot3DWithTXF(cdp_axis, txf_axis, dai_axis, "CDP Rate", "Transaction Fee", "DAI Price")

### Plot DAI Price History with CDP Rate

In [68]:
filename = "test1/sim-summary.pickle"
input_file = open(filename,'rb')
cdp_axis, txf_axis, dai_price_history_axis = pickle.load(input_file)
input_file.close()

In [74]:
plotWithCDP(cdp_axis, txf_axis, dai_price_history_axis, "CDP Rate", "Days", "DAI Price")

### Plot DAI Price History with Tx Fee

In [70]:
plotWithTXF(txf_axis, cdp_axis, dai_price_history_axis, "Tx Fee", "Days", "DAI Price")